### импортируем все нужное

In [4]:
import re
import pandas as pd
import csv
from tqdm import tqdm

In [5]:
try:
    from redditscore.tokenizer_rus import CrazyTokenizer as RusCrazyTokenizer
except:
    print('положите в папку библы redditscore файл tokenizer_rus.py (рядом с обычным tokenizer.py)')

from redditscore.tokenizer import CrazyTokenizer as EngCrazyTokenizer

## Расширяем список стопслов

In [6]:
from nltk.corpus import stopwords

In [7]:
stops = stopwords.words('russian')
stops.extend(stopwords.words('english'))

In [8]:
badwords = [
'я', 'а', 'да', 'но', 'тебе', 'мне', 'ты', 'и', 'у', 'на', 'ага',
'так', 'там', 'какие', 'который', 'какая', 'туда', 'давай', 'короче', 'кажется', 'вообще',
'ну', 'чет', 'неа', 'свои', 'наше', 'хотя', 'такое', 'например', 'кароч', 'как-то',
'нам', 'хм', 'всем', 'нет', 'да', 'оно', 'своем', 'про', 'вы', 'тд', 'тп', 'т.д', 'т.п',
'вся', 'вам', 'это', 'эта', 'эти', 'этот', 'прям', 'либо', 'как', 'мы',
'просто', 'блин', 'очень', 'самые', 'твоем', 'ваша', 'кстати', 'вроде', 'типа', 'пока', 'ок',
    '°', '͜ʖ', '͡', 'ツ', 'ʕ•ᴥ•ʔ'
]

In [9]:
stops.extend(badwords)
stops.remove('не')
stops.remove('нет')
stops.remove('есть')

In [10]:
print(stops)

['и', 'в', 'во', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'сво

## Расширяем список мер

чтоб не лезть руками в код спейси делаем чуть более костыльно

In [11]:
from spacy.lang.char_classes import UNITS
units = UNITS.split('|')
for u in 'кг|г|мг|мкг|л|мл|гр'.split('|'):
    if u not in units:
        units.append(u)
        
print(units)

['km', 'km²', 'km³', 'm', 'm²', 'm³', 'dm', 'dm²', 'dm³', 'cm', 'cm²', 'cm³', 'mm', 'mm²', 'mm³', 'ha', 'µm', 'nm', 'yd', 'in', 'ft', 'kg', 'g', 'mg', 'µg', 't', 'lb', 'oz', 'm/s', 'km/h', 'kmh', 'mph', 'hPa', 'Pa', 'mbar', 'mb', 'MB', 'kb', 'KB', 'gb', 'GB', 'tb', 'TB', 'T', 'G', 'M', 'K', '%', 'км', 'км²', 'км³', 'м²', 'м³', 'дм', 'дм²', 'дм³', 'см', 'см²', 'см³', 'мм', 'мм²', 'мм³', 'нм', 'кг', 'г', 'мг', 'м/с', 'км/ч', 'кПа', 'Па', 'мбар', 'Кб', 'КБ', 'кб', 'Мб', 'МБ', 'мб', 'Гб', 'ГБ', 'гб', 'Тб', 'ТБ', 'тбكم', 'كم²', 'كم³', 'م', 'م²', 'م³', 'سم', 'سم²', 'سم³', 'مم', 'مم²', 'مم³', 'كم', 'غرام', 'جرام', 'جم', 'كغ', 'ملغ', 'كوب', 'اكواب', 'мкг', 'л', 'мл', 'гр']


### Открываем данные

In [12]:
with open('../Data/task2_ru_training.tsv', 'r', encoding='utf-8') as f:
    df = pd.read_csv(f, sep="\t")

In [13]:
df.head(5)

tweet_id                                              tweet  \
0   760402871867367424  Настало время для ингаляторов. Дружок, Сальбут...   
1  1035908416869462016  15) На прошлой зимней олимпиаде большинство лы...   
2  1089839736427032577  Не соглашусь с заменой ЗОК на метопролол в так...   
3   779671488748224513  @di2m1 мезим Смекта Если отравление, то лоперамид   
4   738309299756240897  Уберите микроволновки и имодиум  Действуют соу...   

   class  
0      0  
1      1  
2      0  
3      0  
4      0

### выбираем тестовые твиты

In [21]:
test_data = [df['tweet'][8],
             df['tweet'][9], # дефисы
             df['tweet'][19],
             df['tweet'][268], # дефисы, капс
             
             df['tweet'][273], # латиница, нет пробелов вокруг скобки
             
             df['tweet'][10], # 0,5г ; р/д
             df['tweet'][20], # 125 мгк 2 раза ; Вентолин 100 мгк
            
             df['tweet'][2368], # нет пробела между предложениями
             
             df['tweet'][14], # кавычки, юзернейм, эмоджи
             df['tweet'][18], # хештеги, юзернейм
             df['tweet'][25], # смайл, юзернейм, эмоджи
             df['tweet'][116], # эмоджи
             df['tweet'][270], # смайл
             
             df['tweet'][232] # сокращенная ссылка
            ]

In [22]:
#for tweet in test_data:
#    print(tweet)

## Настраиваем токенизатор

тут версия на улучшенном русском спейси [отсюда](https://github.com/aatimofeev/spacy_russian_tokenizer)

In [23]:
tokenizer_rus = RusCrazyTokenizer(lowercase=True, # к нижнему регистру
                           keepcaps=True, # все кроме all капс
                           urls='ЮРЛ', # удаляем ссылки
                           twitter_handles='ЮЗЕРНЕЙМ', # заменяем все юзернеймы
                           hashtags='ХЕШТЕГ', # пытаемся делить хештеги на слова (тут надо свой частотник закинуть)
                           pos_emojis=True, neg_emojis=True, neutral_emojis=True, # меняем эмоджи на плейсхолдеры (кастомизировать)
                           normalize=3, # удаляем больше 3 одинаковых знаков подряд
                           remove_punct=True, # удаляем пунктуацию
                           remove_breaks=True, # удаляем переносы строк
                           ignore_stopwords=stops, # удаляет стоп слова (разобраться почему падает на кастомном списке)
                           numbers = False,
                           extra_patterns = False
                          )

это на оригинальном английском (если не запускается русская)

In [24]:
tokenizer_eng = EngCrazyTokenizer(lowercase=True, # к нижнему регистру
                           keepcaps=True, # все кроме all капс
                           urls='ЮРЛ', # удаляем ссылки
                           twitter_handles='ЮЗЕРНЕЙМ', # заменяем все юзернеймы
                           hashtags='ХЕШТЕГ', # пытаемся делить хештеги на слова (тут надо свой частотник закинуть)
                           pos_emojis=True, neg_emojis=True, neutral_emojis=True, # меняем эмоджи на плейсхолдеры (кастомизировать)
                           normalize=3, # удаляем больше 3 одинаковых знаков подряд
                           remove_punct=True, # удаляем пунктуацию
                           remove_breaks=True, # удаляем переносы строк
                           ignore_stopwords=stops, # удаляет стоп слова (разобраться почему падает на кастомном списке)
                           numbers = False,
                           extra_patterns = False
                          )

In [25]:
#vocab_rus = list(tokenizer_rus._nlp.vocab.strings)
#vocab_eng = list(tokenizer_eng._nlp.vocab.strings)

## Добавляем еще несколько удалений и замен

In [26]:
def process(tweet, log = False, lang = 'eng') -> str:
    tweet = re.sub('(\w)\.([А-ЯЁ])', r'\1. \2', tweet) # ставим пробелы
    tweet = re.sub('-', ' ', tweet) # удаляем меняем деффисы на пробел
    if log:
        #print('!до токенизации')
        print(tweet)
    # токенизируем (см. удаления и замены выше)
    if lang == 'rus':
        tokens = tokenizer_rus.tokenize(tweet)
    elif lang == 'eng':
        tokens = tokenizer_eng.tokenize(tweet)
    else:
        raise KeyError('ты долбаеб? языка {} нет'.format(lang))
        
    if log:
        #print('!после токенизации')
        print(tokens)
        
    new_tokens = []
    for token in tokens:
        if re.search('\d+', token):
            try:
                x = re.findall('[^\d.,\s]+', token)[0]
                if x in units:
                    token = 'NUM MEASURE'
                else:
                    continue
            except:
                token = 'NUM'
        if re.search('[!"#$%&\'()*+/:;<=>?@[\]^_`{|}~]', token): #удаляем оставшиеся символы
            token = re.sub('[!"#$%&\'()*+/:;<=>?@[\]^_`{|}~]', '', token)
        
        if token in units:
            token = 'MEASURE'
        
        if len(token) < 2:
            continue
            
        new_tokens.append(token)
            
    if log:
        #print('!после обработки каждого токена')
        print(new_tokens)
        #print('!output')
    
    return ' '.join(new_tokens)

In [27]:
res = []
for tweet in test_data:
    print(tweet)
    tweet = process(tweet, log=False, lang='rus')
    res.append(tweet)
    print(tweet, '\n')

Все эти слюнтяи, утверждающие, будто суицид-удел слабых людей, найдут тысячу причин не глотать горсть прозака.Им просто не хватает смелости.
слюнтяи утверждающие суицид удел слабых людей найдут тысячу причин не глотать горсть прозака не хватает смелости 

Это вам не лыжи,сальбутамол не поможет. Россия - Норвегия 5:1
не лыжи сальбутамол не поможет россия норвегия 

Если оланзапин мне не поможет, тогда мне назначат галоперидол. Не хочу докатываться до галоперидола. Надо что-то делать с этой проклятой тревожностью.
оланзапин не поможет назначат галоперидол не хочу докатываться галоперидола делать проклятой тревожностью 

из-за флуоксетина катастрофически не хочется что-либо есть, но почему-то БЛИНЫ ИЗ ТЕРЕМКА в меня влезают всегда  Теремок, спасибо что живая
флуоксетина катастрофически не хочется есть почему БЛИНЫ ТЕРЕМКА влезают теремок спасибо живая 

В моей голове Хенвон любит lil peep; перкосет, молли, перпл дранк(это все наркота, если кто не в курсе); организует нелегальные вечеринки

In [28]:
res

['слюнтяи утверждающие суицид удел слабых людей найдут тысячу причин не глотать горсть прозака не хватает смелости',
 'не лыжи сальбутамол не поможет россия норвегия',
 'оланзапин не поможет назначат галоперидол не хочу докатываться галоперидола делать проклятой тревожностью',
 'флуоксетина катастрофически не хочется есть почему БЛИНЫ ТЕРЕМКА влезают теремок спасибо живая',
 'моей голове хенвон любит lil peep перкосет молли перпл дранк наркота не курсе организует нелегальные вечеринки',
 'ципрофлоксацин NUM NUM MEASURE NUM смысл жизни',
 'ветеринар прописал ингаляции фликсотидом NUM мгк NUM раза сутки прописал вентолин NUM мгк случай приступов',
 'день выпила NUM таблеток алпразолама чувствую расслабленно ох днем часа NUM спала',
 'ЮЗЕРНЕЙМ не лечатся футболисты астмы синдрома дефицита внимания столь любимыми нероссийских спортсменов законными амфетамином декстроамфетамином',
 'ЮЗЕРНЕЙМ порекомендовал сотрудникам wada выпить мельдония вдохнуть сальбутамола попробовать приблизиться резу

## тут можно тестировать конкретные случаи и писать о  траблах ане

In [29]:
test_phrase = 'я вдолбил 100г мефа и 20мл пива 50л колы и 18кг моркови а еще потом занюхал 18мг герыча'
log = False
lang = 'rus'

In [30]:
test_tokenizer = EngCrazyTokenizer(lowercase=True, # к нижнему регистру
                           keepcaps=True, # все кроме all капс
                           urls='ЮРЛ', # удаляем ссылки
                           twitter_handles='ЮЗЕРНЕЙМ', # заменяем все юзернеймы
                           hashtags='ХЕШТЕГ', # пытаемся делить хештеги на слова (тут надо свой частотник закинуть)
                           pos_emojis=False, neg_emojis=False, neutral_emojis=False, # меняем эмоджи на плейсхолдеры (кастомизировать)
                           normalize=3, # удаляем больше 3 одинаковых знаков подряд
                           remove_punct=True, # удаляем пунктуацию
                           remove_breaks=True, # удаляем переносы строк
                           ignore_stopwords=False, # удаляет стоп слова (разобраться почему падает на кастомном списке)
                           numbers = False,
                           extra_patterns = False
                          )

In [31]:
print('before  :', test_phrase)
res_phrase = process(test_phrase, log = log, lang = lang)
print('after   :',res_phrase)

print('del/repl:', ', '.join([w for w in test_tokenizer.tokenize(test_phrase) if w not in res_phrase.split(' ')]))

before  : я вдолбил 100г мефа и 20мл пива 50л колы и 18кг моркови а еще потом занюхал 18мг герыча
after   : вдолбил NUM MEASURE мефа NUM MEASURE пива NUM MEASURE колы NUM MEASURE моркови занюхал NUM MEASURE герыча
del/repl: я, 100, г, и, 20мл, 50л, и, 18, кг, а, еще, потом, 18, мг


## Лемматизация

### nltk snowball stemmer

In [32]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

In [33]:
for tweet in res:
    print(tweet)
    print(' '.join([stemmer.stem(w) for w in tweet.split(' ')]), '\n')

слюнтяи утверждающие суицид удел слабых людей найдут тысячу причин не глотать горсть прозака не хватает смелости
слюнтя утвержда суицид удел слаб люд найдут тысяч причин не глота горст прозак не хвата смелост 

не лыжи сальбутамол не поможет россия норвегия
не лыж сальбутамол не поможет росс норвег 

оланзапин не поможет назначат галоперидол не хочу докатываться галоперидола делать проклятой тревожностью
оланзапин не поможет назначат галоперидол не хоч докатыва галоперидол дела проклят тревожн 

флуоксетина катастрофически не хочется есть почему БЛИНЫ ТЕРЕМКА влезают теремок спасибо живая
флуоксетин катастрофическ не хочет ест поч блин теремк влеза теремок спасиб жив 

моей голове хенвон любит lil peep перкосет молли перпл дранк наркота не курсе организует нелегальные вечеринки
мо голов хенвон люб lil peep перкосет молл перпл дранк наркот не курс организ нелегальн вечеринк 

ципрофлоксацин NUM NUM MEASURE NUM смысл жизни
ципрофлоксацин NUM NUM MEASURE NUM смысл жизн 

ветеринар прописа

### Mystem

In [86]:
from pymystem3 import Mystem
mstm = Mystem()

In [90]:
lem = []
for tweet in tqdm(res):
    lem.append(''.join(mstm.lemmatize(tweet)).strip())

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:11<00:00,  1.24it/s]


In [91]:
lem

['слюнтяй утверждать суицид удел слабый человек находить тысяча причина глотать горсть прозак хватать смелость',
 'лыжа сальбутамол помогать россия норвегия',
 'оланзапина помогать назначать галоперидол хотеть докатываться галоперидол делать проклятый тревожность',
 'флуоксетин катастрофически хотеться почему блин теремок влезать теремок спасибо живой',
 'мой голова хенвон любить lil peep перкосать молль перпл дранк наркота курс организовывать нелегальный вечеринка',
 'ципрофлоксацин смысл жизнь',
 'ветеринар прописывать ингаляция фликсотид мгк раз сутки прописывать вентолина мгк случай приступ',
 'день выпивать таблетка алпразолам чувствовать расслабленно ох день час спать',
 'юзернейм лечиться футболист астма синдром дефицит внимание столь любимый нероссийский спортсмен законный амфетамин декстроамфетамин',
 'юзернейм порекомендовать сотрудник wada выпивать мельдония вдыхать сальбутамол попробовать приближаться результат любой медалист splits вад крушельницкий',
 'юзернейм питер мема

### pymorphy

# Полный скрипт препроцессинга

In [34]:
def write_data(out_file, prep = True, lemmatize = None):
    with open('../Data/task2_ru_training.tsv', 'r', encoding='utf-8') as f:
        df = pd.read_csv(f, sep="\t")
    
    if prep:
        new_tweets = []
        for tweet in tqdm(df['tweet']):
            new_tweets.append(process(tweet, log=False, lang = 'rus'))
        df['tweet'] = new_tweets
        
    if lemmatize == 'mystem':
        lem = []
        for tweet in tqdm(df['tweet']):
            lem.append(''.join(mstm.lemmatize(tweet)).strip())
        df['tweet'] = lem
    elif lemmatize == 'snowball':
        lem = []
        for tweet in tqdm(df['tweet']):
            lem.append(' '.join([stemmer.stem(w).strip() for w in tweet.split(' ')]))
        df['tweet'] = lem
    elif lemmatize == 'pymorphy':
        pass
    else:
        pass


    with open('../Data/{}'.format(out_file), 'w', encoding='utf-8') as output:
        df.to_csv(output, sep='\t', index=False, line_terminator='\n')

    print('Done! Your data is in {}'.format(out_file))

In [35]:
out_file = 'snowball_lem.tsv'

In [36]:
write_data(out_file, prep = True, lemmatize = 'snowball')

100%|█████████████████████████████████████████████████████████████████████████████| 6090/6090 [00:08<00:00, 739.26it/s]


Done! Your data is in snowball_lem.tsv
